This is a notebook to solve linear programming problem using the simmplex method. The inputs are supplied in the form of standard LPP that is the objective shooulf be of maximization type and all the constraints should be of <= type.
Example being:

maximize z: 3*x1 + 4*x2

      s.t.: 5*x1+ 4*x2<=15
      ...
      
Here a sample problem is solved:
Maximize	P	=	20x1	+	10x2	+	15x3	


Subject to: 

3x1	+	2x2	+	5x3	≤	55

2x1	+	x2	+	x3	≤	26

x1	+	x2	+	3x3	≤	30

5x1	+	2x2	+	4x3	≤	57

x1	,	x2	,	x3	≥	0


In [1]:
import numpy as np

# Enter data for the simplex:

In [2]:
num_vars = None # 3 here
cost_vector = None #[20 10 15]
num_consts = None # 4
A = None #[[3 2 5],[2 1 1], [1 1 3],[5 2 4]]
b = None #[55 26 30 57]

In [3]:
print("Enter the Linear programming data in standard form: ")
num_vars = int(input('Number of variables: '))

cost_vector = input('Coefficients of the decision variables in cost function: ')
cost_vector = cost_vector.split(' ')
cost_vector = [int(i) for i in cost_vector]
cost_vector = np.array(cost_vector)

num_consts = int(input('Number of constraints: '))
print("Inequality constraints data: ")

print('Enter the matrix of coefficients: ')

A = np.zeros((num_consts,num_vars))

for i in range(num_consts):
    print("Constraint: "+str(i+1))
    x = input("Coefficients: ")
    x = x.split(' ')
    x = [float(k) for k in x]
    for j in range(num_vars):
        A[i][j] = x[j]


b = input("Enter the RHS: ")
b = b.split(' ')
b = [float(k) for k in b]

Enter the Linear programming data in standard form: 
Number of variables: 3
Coefficients of the decision variables in cost function: 20 10 15
Number of constraints: 4
Inequality constraints data: 
Enter the matrix of coefficients: 
Constraint: 1
Coefficients: 3 2 5
Constraint: 2
Coefficients: 2 1 1
Constraint: 3
Coefficients: 1 1 3
Constraint: 4
Coefficients: 5 2 4
Enter the RHS: 55 26 30 57


In [4]:
A_ = A.copy()
b_ = b.copy()

# Enter again from here on the same data:

In [5]:
A = A_.copy()
b = b_.copy()

In [6]:
num_le = num_consts
num_eq = 0
num_ge = 0



v_slack = []
A_slacks = np.zeros((num_consts,num_consts))
v_artificial = []
A_surplus = np.zeros((num_consts,num_ge))
v_bounds = []



for i in  range(num_consts):
    s = '<='

    if s=='<=':
        v_slack.append(b[i])
        A_slacks[i][len(v_slack)-1] = 1
    elif s=='>=':
        v_slack.append(0)
        A_slacks[i][len(v_slack)-1] = -1
        v_artificial.append(b[i])
        A_surplus[i][len(v_artificial)-1] = 1
    else:
        v_artificial.append(b[i])
        A_surplus[i][len(v_artificial)-1] = 1
    
    v_bounds.append(b[i])


A = np.hstack([A,A_slacks,A_surplus])

vari = []
vari_ar = []
vari_slack = []
vari_nb = []

variables = []
for i in  range(len(A[0])):
    variables.append('x'+str(i+1))
    vari.append('x'+str(i+1))
    if i < num_vars:
        vari_nb.append('x'+str(i+1))
    elif i< num_vars + len(v_slack):
        vari_slack.append('x'+str(i+1))
    else:
        vari_ar.append('x'+str(i+1))

all_vars = {}

In [7]:
v_a = 0*cost_vector
v_ar = None

x = np.hstack([v_a,v_slack,v_artificial])

for v,val in zip(variables,x):
    all_vars[v] = val


if len(v_artificial)==0:
    v_ar = []
else:
    if type_problem == 1:
        v_ar = -9999*np.ones(len(v_artificial))
    else:
        v_ar = 9999*np.ones(len(v_artificial))

Cj = np.hstack([cost_vector,0*np.array(v_slack),v_ar])
Ci = []
tab1 = []
Vb = []
Q = v_bounds

struct2_curr_values = np.zeros(len(Q))
struct2_var_base = ['' for _ in range(len(Q))]

for i in range(len(Q)):
    tab1.append('|')
    struct2_curr_values = Q[i]
    ind = np.where(x==Q[i])
    ind = ind[0]
    ind = ind[0]

    struct2_var_base[i] = variables[ind]
    Vb.append(struct2_var_base[i])
    Ci.append(Cj[ind])

In [8]:
Ci = np.array(Ci)

Z = sum(np.multiply(Ci,Q))
Zj = np.zeros(len(Cj))

for i in range(len(Cj)):
    Zj[i] = sum(np.multiply(Ci,A[:,i])) 
vars_at_moment = struct2_var_base.copy()

In [9]:
def get_row(vec):
    k = ''
    for i in vec:
        k+= '{0:>10}'.format(round(i,3))
    return k

In [10]:
print("=========LP in standard form:=========")

print("variables: ",vari)
print("Activity variables: ",vari_nb)
print("Slack variables: ",vari_slack)
print("Artificial variables: ",vari_ar)

print("======Iteration: 0======")
print("Initializing variables: ",vari)
print("Activity variables: ",[all_vars[v] for v in vari_nb])
print("Slack variables: ",vari_slack,[all_vars[v] for v in vari_slack])
print("Artificial variables: ",v_ar)
print('=======================================================================================================')
print("Variables:                              ",get_row(np.arange(start=1,stop=len(A[0,:])+1,step = 1 )))
print("Cj:                                     ",get_row(Cj))
print("Basic        |Value       |RHS")
for i in range(num_consts):
    print('|',"{0:>10}".format(vars_at_moment[i]),'|',"{0:>10}".format(Ci[i]),'|',"{0:>10}".format(round(Q[i],3)),'|',get_row(A[i]),'|')

print('=======================================================================================================')
print('Zj:                                     ',get_row(Zj))
print('Cj-Zj:                                  ',get_row(Cj-Zj))
print('Z: ',round(Z,3))

=========LP in standard form:=========
variables:  ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7']
Activity variables:  ['x1', 'x2', 'x3']
Slack variables:  ['x4', 'x5', 'x6', 'x7']
Artificial variables:  []
======Iteration: 0======
Initializing variables:  ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7']
Activity variables:  [0.0, 0.0, 0.0]
Slack variables:  ['x4', 'x5', 'x6', 'x7'] [55.0, 26.0, 30.0, 57.0]
Artificial variables:  []
Variables:                                        1         2         3         4         5         6         7
Cj:                                            20.0      10.0      15.0       0.0       0.0       0.0       0.0
Basic        |Value       |RHS
|         x4 |        0.0 |       55.0 |        3.0       2.0       5.0       1.0       0.0       0.0       0.0 |
|         x5 |        0.0 |       26.0 |        2.0       1.0       1.0       0.0       1.0       0.0       0.0 |
|         x6 |        0.0 |       30.0 |        1.0       1.0       3.0       0.0       0.

In [11]:
iterNum = 1

In [12]:

while iterNum<=10:
    iterNum+=1
    type_problem = 1
    if type_problem == 1:
        num = max(Cj-Zj)
        index = np.where((Cj-Zj)==num)
        index = index[0][0]
        v_enter = variables[index]
    else:
        num = min(Cj-Zj)
        index = np.where((Cj-Zj)==num)
        index = index[0][0]
        v_enter = variables[index]
    
    b = A[:,index]
    k = -1
    d = 10000
    
    for i in range(len(Q)):
        if b[i]>0:
            div = Q[i]/b[i]
            if d>=div:
                d = div
                k = i
    if k == -1:
        print('Solution is infinty ')
        break
    else:
        num2 = k
    
    v_leave = struct2_var_base[num2]
    struct2_var_base[num2] = v_enter
    pivot = A[num2,index]
    Ci[num2] = Cj[index]
    A[num2,:] = A[num2,:]/pivot
    Q[num2] = Q[num2]/pivot
    
    #Row operations:
    
    for i in range(num_consts):
        
        if i!= num2:
            Q[i] = Q[i] - A[i,index]*Q[num2]
            A[i,:] = A[i,:] - A[i,index]*A[num2,:]
    Z = sum(np.multiply(Ci,Q))
    for i in  range(len(A[0,:])):
        Zj[i] = sum(np.multiply(A[:,i],Ci))
    
    vars_at_moment = []
    for i in range(num_consts):
        vars_at_moment.append(struct2_var_base[i])
    
    print('=====Iteration',iterNum,'=======')
    print('Enter: ',v_enter)
    print('Leave: ',v_leave)
    print('Pivot: ',round(pivot,3))
    
    print('========')

    print('=======================================================================================================')
    print("Variables:                              ",get_row(np.arange(start=1,stop=len(A[0,:])+1,step = 1 )))
    print("Cj:                                     ",get_row(Cj))
    print("Basic        |Value       |RHS")
    for i in range(num_consts):
        print('|',"{0:>10}".format(vars_at_moment[i]),'|',"{0:>10}".format(Ci[i]),'|',"{0:>10}".format(round(Q[i],3)),'|',get_row(A[i]),'|')

    print('=======================================================================================================')
    print('Zj:                                     ',get_row(Zj))
    print('Cj-Zj:                                  ',get_row(Cj-Zj))
    print('Z: ',round(Z,3))
    if type_problem ==1:
        temp = max(Cj-Zj)
        if temp<=0:
            break
    else:
        temp = min(Cj-Zj)
        if temp>=0:
            break
    

=====Iteration 2 =======
Enter:  x1
Leave:  x7
Pivot:  5.0
Variables:                                        1         2         3         4         5         6         7
Cj:                                            20.0      10.0      15.0       0.0       0.0       0.0       0.0
Basic        |Value       |RHS
|         x4 |        0.0 |       20.8 |        0.0       0.8       2.6       1.0       0.0       0.0      -0.6 |
|         x5 |        0.0 |        3.2 |        0.0       0.2      -0.6       0.0       1.0       0.0      -0.4 |
|         x6 |        0.0 |       18.6 |        0.0       0.6       2.2       0.0       0.0       1.0      -0.2 |
|         x1 |       20.0 |       11.4 |        1.0       0.4       0.8       0.0       0.0       0.0       0.2 |
Zj:                                            20.0       8.0      16.0       0.0       0.0       0.0       4.0
Cj-Zj:                                          0.0       2.0      -1.0       0.0       0.0       0.0      -4.0
Z:  22